In [ ]:
# Transfer Learning re training the last layers

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing import image
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

# variable initiallization

batch_size = 128
num_classes = 10
epochs = 12


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# reduce the data
x_train_red = x_train[0:100]
x_test_red = x_test[0:16]
y_train_red = y_train[0:100]
y_test_red = y_test[0:16]

In [ ]:
x_train_red.shape

In [ ]:
# resize the data set

import numpy as np
xtrain1 = [np.resize(x_train_red[i], (200, 200, 3)) for i in range(0, len(x_train_red))] 
x_train_new = np.array(xtrain1)

In [ ]:
x_train_new.shape

In [ ]:
# resize the data set

xtest1 = [np.resize(x_test_red[i], (200, 200, 3)) for i in range(0, len(x_test_red))] 
x_test_new = np.array(xtest1)

In [ ]:
x_test_new.shape

In [ ]:
# data rescaling  

x_train_new = x_train_new.astype('float32')
x_test_new = x_test_new.astype('float32')
x_train_new /= 255
x_test_new /= 255
print('x_train shape:', x_train_new.shape)
print(x_train_new.shape[0], 'original train samples')
print(x_test_new.shape[0], 'original test samples')



In [ ]:
x_train_new.shape

In [ ]:
# Creating bottle neck features

from keras.models import Model

# create the base pre-trained model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(200,200,3))
#Preprocessing the data, so that it can be fed to the pre-trained ResNet50 model. 
resnet_train_input = preprocess_input(x_train_new)

#Creating bottleneck features for the training data
train_features = model.predict(resnet_train_input)

print("Building Features")

In [ ]:
resnet_train_input.shape

In [ ]:
train_features.shape

In [ ]:
# Compile the model

model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=train_features.shape[1:]))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))
model.summary()

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional ResNet50 layers
for layer in model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print("compiled successfully")

In [ ]:
resnet_train_input.shape

In [ ]:
#Train the model

model.fit(train_features, y_train_red,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)


In [ ]:
# score the model
score = model.evaluate(train_features, y_train_red)

In [ ]:
# check the loss and accuracy
score

In [ ]:
# Predict 
model.predict(train_features[1:2])

In [ ]:
# what was it ?
y_train_red[1:2]